In [ ]:
import time
import random
import os
import numpy as np
import pandas as pd

CLASSES_NUM = 10

def get_sampled_classes(sample_num=CLASSES_NUM):
    random.seed(2023)
    train_path = '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train'
    classes_name = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
    print('len(classes_name): ', len(classes_name))
    classes_name = random.sample(classes_name, sample_num)
    return classes_name

SAMPLED_CLASSES = get_sampled_classes(sample_num=CLASSES_NUM)
print('SAMPLED_CLASSES:', SAMPLED_CLASSES[:10])
# SAMPLED_CLASSES = ['n02325366', 'n03938244', 'n04086273', 'n03393912', 'n02804610', 'n02268443', 'n03697007', 
#                    'n03240683', 'n04067472', 'n04536866', 'n02093991', 'n03388549', 'n02108551', 'n03791053', 
#                    'n04149813', 'n02099712', 'n07583066', 'n02747177', 'n01608432', 'n03876231']

SAMPLED_CLASSES = set(SAMPLED_CLASSES)
# len(classes_name):  1000
# SAMPLED_CLASSES: ['n02325366', 'n03938244', 'n04086273', 'n03393912', 'n02804610', 
#                   'n02268443', 'n03697007', 'n03240683', 'n04067472', 'n04536866']

In [ ]:
class_mapping_file = '/kaggle/input/imagenet-object-localization-challenge/LOC_synset_mapping.txt'
class_mapping_dict = {}
for line in open(class_mapping_file):
    if line[:9] in SAMPLED_CLASSES:
        real_class_name = line[10:].strip()
        class_mapping_dict[line[:9]] = real_class_name
        
print('class_mapping_dict: ', class_mapping_dict)

In [ ]:
train_path = '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train/'
new_train_path = '/kaggle/working/training/'
os.system(f'rm -rf {new_train_path}')
os.system(f'mkdir {new_train_path}')

for i, name in enumerate(SAMPLED_CLASSES):
    print('i: ', i)
    os.system(f'cp -r {train_path + name}  {new_train_path}')
print('get here!!!')

In [ ]:
valid_path = '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val/'
new_valid_path = '/kaggle/working/validation/'
os.system(f'rm -rf {new_valid_path}')
os.system(f'mkdir {new_valid_path}')

process_start_t = time.time()

os.chdir('/kaggle/working/validation/')

line_cnt, write_cnt = 0, 0
for line in open('/kaggle/input/valprep/valprep.sh'):
    line_cnt += 1
    if line_cnt % 10000 == 0:
        print('line_cnt: ', line_cnt)
    if line.startswith('mkdir'):
        class_name = line.strip().split(' ')[-1]
        if class_name in SAMPLED_CLASSES:
            write_cnt += 1
            #file_w.write(line)
            os.system(line.strip())
    elif line.startswith('mv'):
        class_name = line.strip().split(' ')[-1][:-1]
        if class_name in SAMPLED_CLASSES:
            write_cnt += 1
            src_file_name = line.strip().split(' ')[1]
            #file_w.write(f'cp {valid_path}{src_file_name} {class_name}/' + '\n')
            os.system(f'cp {valid_path}{src_file_name} {class_name}/')
        
print('ok, get here!!! write_cnt: ', write_cnt)

os.chdir('/kaggle/working/')

print(f'get here!!! cost time: {time.time()-process_start_t:.2f} sec')

In [ ]:
def get_all_files(directory):
    filenames = []
    class_name = directory.split('/')[-1]
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            filenames.append(filename)
    return filenames

for i, directory in enumerate(SAMPLED_CLASSES):
    directory_path = new_valid_path + '/' + directory
    filenames = get_all_files(directory_path)
    print(f'valid class_name: {directory}', len(filenames))
    
    
for i, directory in enumerate(SAMPLED_CLASSES):
    directory_path = new_train_path + '/' + directory
    filenames = get_all_files(directory_path)
    print(f'train class_name: {directory}', len(filenames))

In [ ]:
!du -sh ./training

In [ ]:
zip_start_t = time.time()
!zip -r -q training.zip ./training
!zip -r -q validation.zip ./validation

print(f'ok, get here!!! zip cost time: {time.time()-zip_start_t:.2f} sec')

In [ ]:
!rm -rf ./training
!rm -rf ./validation